# 📈 AI Stock Price Prediction
## Using Linear Regression to Predict Stock Prices

In [ ]:
# Install required packages
!pip install yfinance scikit-learn pandas numpy matplotlib streamlit

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Get stock data
def get_stock_data(symbol, period="1y"):
    stock = yf.Ticker(symbol)
    data = stock.history(period=period)
    return data

# Get Tesla data
tesla_data = get_stock_data("TSLA")
print(f"Data shape: {tesla_data.shape}")
tesla_data.head()

In [ ]:
# Prepare features
def prepare_features(data):
    df = data.copy()
    df['Day'] = range(len(df))
    df['MA_5'] = df['Close'].rolling(5).mean()
    df['MA_10'] = df['Close'].rolling(10).mean()
    df['Volume_MA'] = df['Volume'].rolling(5).mean()
    df = df.dropna()
    
    features = ['Day', 'Open', 'High', 'Low', 'Volume', 'MA_5', 'MA_10', 'Volume_MA']
    X = df[features]
    y = df['Close']
    
    return X, y, df

X, y, processed_data = prepare_features(tesla_data)
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.2f}")
print(f"Accuracy: {r2*100:.1f}%")

In [ ]:
# Plot results
plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title('Tesla Stock Price Prediction')
plt.show()

# Time series plot
plt.figure(figsize=(12, 6))
plt.plot(processed_data.index[-len(y_test):], y_test.values, label='Actual', alpha=0.7)
plt.plot(processed_data.index[-len(y_test):], y_pred, label='Predicted', alpha=0.7)
plt.legend()
plt.title('Tesla Stock Price: Actual vs Predicted')
plt.show()

In [ ]:
# Save model
with open('stock_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved as stock_model.pkl")